# 02b_deep_url_stats
In the previous chapter, `02`, we dove deep into what was going on in the tweets we collected surrounding the world cup. While we found many an insight here, something that particularly stood out was the nature of URLs leaving Twitter, and the seeming popularity of linking to semi-legal/illegal streaming sites. 

In this sub-chapter, we look deeper into this phenomenon, with the goal of producing reliable statistics/distributions of URL sharing patterns in the twitter data chunks we collected surrounding the world cup. 

NL, 08/01/23

### IMPORTS

In [68]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

### PATHS & CONSTANTS

In [5]:
EXPORT_PATH = '/home/nikloynes/projects/world_cup_misinfo_tracking/data/exports/tweets/'

EXPANDED_URLS = EXPORT_PATH+'expanded_urls.json'
ALL_URLS_TIMESTAMPS = EXPORT_PATH+'all_urls_timestamps.csv'
TWITTER_URLS = EXPORT_PATH+'twitter_urls.csv'
NON_TWITTER_URLS = EXPORT_PATH+'non_twitter_urls.csv'
DOMAIN_FREQUENCIES = EXPORT_PATH+'domain_frequencies.csv'

### INIT

In [69]:
pio.renderers.default='notebook'

In [37]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [17]:
all_urls_timestamps_df = pd.read_csv(ALL_URLS_TIMESTAMPS)
twitter_urls_df = pd.read_csv(TWITTER_URLS)
non_twitter_urls_df = pd.read_csv(NON_TWITTER_URLS)
domain_frequencies_df = pd.read_csv(DOMAIN_FREQUENCIES)

with open(EXPANDED_URLS, 'r') as infile:
    expanded_urls = [json.loads(line) for line in infile]
expanded_urls_df = pd.DataFrame(expanded_urls)

### THE THING!

#### 1. Overview / summary stats

In [10]:
len(all_urls_timestamps_df)

1582846

**Overall, there were `1,582,856` tweets containing a URL.**  

In [11]:
len(expanded_urls_df)

562608

**These tweets contained a total of `562,608` unique URLs - most of them shortened to the t.co domain. This means that the true value of unique URLs (i.e. different targets on the web is likely lower, as the same full URL can have different shortened versions.)**

We expanded these shortened URLs. This allows us to see how many URLs in tweets refer to Twitter-objects, and how many of them link outside of Twitter.

Let's now merge in the url-metadata into the non-unique URL sharing table

In [20]:
all_urls_timestamps_df = all_urls_timestamps_df.rename(columns={'url' : 'original_url'})

In [21]:
all_urls_timestamps_df = all_urls_timestamps_df.merge(pd.concat([twitter_urls_df, non_twitter_urls_df]), on='original_url', how='left')

In [24]:
all_urls_timestamps_df.loc[all_urls_timestamps_df['resolved_domain']=='twitter.com', 'internal'] = True

In [26]:
all_urls_timestamps_df['internal'].fillna(False, inplace=True)

In [28]:
all_urls_timestamps_df['internal'].value_counts()

True     1109868
False     472978
Name: internal, dtype: int64

In [29]:
all_urls_timestamps_df['internal'].value_counts(normalize=True)

True     0.701185
False    0.298815
Name: internal, dtype: float64

**Out of all the tweets we've collected, for those containing URLs, `1,109,868` times a URL shared was to a Twitter resource. That represents `~70%` of all URL-sharing occurrences.**

**Out of all the tweets we've collected, for those containing URLs, `472,978` times a URL shared was somewhere outside of twitter. This represents `~30%` of all URL-sharing occurrences.**

In [30]:
len(twitter_urls_df)

444297

In [31]:
len(non_twitter_urls_df)

111969

In [35]:
len(twitter_urls_df)/(len(twitter_urls_df)+len(non_twitter_urls_df))*100

79.87132055527391

In [34]:
len(non_twitter_urls_df)/(len(twitter_urls_df)+len(non_twitter_urls_df))*100

20.12867944472608

When we look at unique shortened URLs (rather than incidences of URL sharing), we have the following distributions:

**For unique URLs linking to Twitter, we have `444,297` unique URLs, which represent `~79.8%` of all shortened URLs**

**For unique URLs linking away from Twitter, we have `111,969` unique URLs, which represent `~20.1%` of all shortened URLs**

#### 2. Domains

Besides having a look at the distribution of domains for links out of Twitter, we also want to make a judgment regarding which domains likely link to illegal streaming sites. 

In order to achieve this, we will manually check some domains unknown to us, and see what we can find. We will limit this to the 100 most frequent expanded domains -- **this means that any resulting stats for this will likely be underestimating the number of streaming-related URL-sharing incidences**

In [38]:
domain_frequencies_df[:100]

,resolved_domain,freq
0,blogspot.com,145000
1,youtube.com,9329
2,abema.tv,9094
3,t.me,8091
4,mtl37dt.com,5204
5,t.,5037
6,t,5028
7,whatsapp.com,4503
8,t.c,4318
9,apple.com,4279


We have found a list of 9 likely (semi)-illegal streaming sites:

It is important to note that we are classing all `blogspot.com` incidences as streaming sites... which surely isn't true.

In [43]:
streaming = [
    'blogspot.com',
    # 'abema.tv', -- wrong, abema.tv is a legit japanese streaming site
    'mtl37dt.com',
    'streamssports.live',
    'directstream24.com',
    'contents-abema.com',
    'megaevent.live',
    'flash-streams.net',
    'bestsports-stream.com',
]

In [47]:
all_urls_timestamps_df.loc[all_urls_timestamps_df['resolved_domain'].isin(streaming), 'is_streaming'] = True

In [50]:
all_urls_timestamps_df['is_streaming'].fillna(False, inplace=True)

In [51]:
all_urls_timestamps_df['is_streaming'].value_counts()

False    1425463
True      157383
Name: is_streaming, dtype: int64

In [52]:
all_urls_timestamps_df['is_streaming'].value_counts(normalize=True)

False    0.90057
True     0.09943
Name: is_streaming, dtype: float64

In [54]:
all_urls_timestamps_df[all_urls_timestamps_df['resolved_domain']!='twitter.com']['is_streaming'].value_counts()

False    315595
True     157383
Name: is_streaming, dtype: int64

In [55]:
all_urls_timestamps_df[all_urls_timestamps_df['resolved_domain']!='twitter.com']['is_streaming'].value_counts(normalize=True)

False    0.667251
True     0.332749
Name: is_streaming, dtype: float64

In [56]:
len(non_twitter_urls_df[non_twitter_urls_df['resolved_domain'].isin(streaming)])

6055

In [58]:
len(non_twitter_urls_df[non_twitter_urls_df['resolved_domain'].isin(streaming)])/len(non_twitter_urls_df)*100

5.407746787057132

So, we have now looked at the proportion of streaming links in our entire sample of tweets. 

- Out of our likely undercounting list of domains which we've defined as illegal streaming sites, we found that at **`6,055`** unique URLs refer to such sites. That is approximately **`~5%`** of all unique non-Twitter URLs.

- We find that out of all URL-sharing incidences in our sample of tweets, **at least `157,383`** are to a likely streaming site. This represents **just under 10% of all URL shares** in our sample, regardless of to Twitter or away from Twitter

- If we just consider URL sharing away from Twitter, we find that at the `157,383` incidences represent **around `33%`** all URL shares in our sample.

#### 3. Streaming URL sharing patterns over time?

Finally, a little bonus. We might want to know how sharing of links to streaming sites evolved over time. 

In [65]:
all_urls_timestamps_df['timestamp'] = pd.to_datetime(all_urls_timestamps_df['timestamp'])

In [66]:
hourly_df = all_urls_timestamps_df.groupby(pd.Grouper(key='timestamp', freq='60Min')).mean().reset_index()

/tmp/ipykernel_299282/1727822508.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hourly_df = all_urls_timestamps_df.groupby(pd.Grouper(key='timestamp', freq='60Min')).mean().reset_index()


In [73]:
hourly_df['is_streaming'] = np.round(hourly_df['is_streaming']*100, decimals=2)

In [74]:
dates = ['2022-11-26', '2022-12-04', '2022-12-14', '2022-12-18']

date_ranges = [
    ['2022-11-26 03:00', '2022-11-26 14:00'],
    ['2022-12-04 16:00', '2022-12-04 23:30'],
    ['2022-12-14 16:00', '2022-12-14 23:30'],
    ['2022-12-18 14:00', '2022-12-18 20:30']
]

colours = ['blue', 'purple', 'orange', 'green']

for date in date_ranges: 
    subset_df = hourly_df.query(f'timestamp.between("{date[0]}", "{date[1]}")')

    fig = go.Figure()
    fig.add_scatter(name='% streaming links', x=subset_df['timestamp'], y=subset_df['is_streaming'])
    fig.update_traces(connectgaps=True)
    fig.update_layout(
        title=f'% of URL shares to streaming sites, on {date[0][:10]}',
        xaxis_title='Tweet posted at',
        yaxis_title='Percentage'
    )
    fig.show()

So, sharing patterns to streaming sites do seem to track the kickoff times of games. 

Drop offs, especially on the day of the final, and to ZERO percent, are explained by no tweets being available in our sample. 